In [98]:
import pandas as pd # 데이터 전처리
import numpy as np # 데이터 전처리
import matplotlib.pyplot as plt # 데이터 시각화

import warnings
from tqdm import tqdm
from datetime import datetime, timedelta # 시간 데이터 처리

from statsmodels.tsa.arima_model import ARIMA # ARIMA 모델
from fbprophet import Prophet

warnings.filterwarnings('ignore')
%matplotlib inline

In [99]:
def fill_nan(df):
    for col in df.columns[1:]:
        start_dt = df.loc[df.loc[:, col].isnull() == False].index[0]
        from_dt = start_dt
        from_val = df.loc[start_dt, col]

        for dt, val in df.loc[start_dt:, col].items():
            if not pd.isna(val):
                # processing
                length = len(df.loc[from_dt: dt, col]) - 1
                if length != 0:
                    for i in df.loc[from_dt: dt, col].index:
                        df.loc[i, col] = from_val / length

                from_dt = dt
                from_val = val
    return df


def to_panel_format(df):
    place_id=[]; time=[] ; target=[] # 빈 리스트를 생성합니다.
    
    df['Time'] = pd.to_datetime(df['Time']) 
    df = df.set_index('Time')
    
    for i in df.columns:
        for j in range(len(df)):
            place_id.append(i) # place_id에 미터 ID를 정리합니다.
            time.append(df.index[j]) # time에 시간대를 정리합니다.
            target.append(df[i].iloc[j]) # target에 전력량을 정리합니다.

    ret = pd.DataFrame({'place_id':place_id,'time':time,'target':target})
    ret = ret.dropna() # 결측치를 제거합니다.
    ret = ret.set_index('time') # time을 인덱스로 저장합니다.
    return ret


def get_optimal_params(y):
    param_dict = {}
    
    for param in pdq:
        try:
            model = ARIMA(y, order=param)
            results_ARIMA = model.fit(disp=-1)
            param_dict[results_ARIMA.aic] = param
        except:
            continue

    min_aic = min(param_dict.keys())
    optimal_params = param_dict[min_aic]
    return optimal_params

def prediction(df):
    p = d = q = range(0, 2)
    pdq = list(itertools.product(p, d, q))

    ret = {}
    sub = pd.DataFrame() # a라는 데이터프레임에 예측값을 정리합니다.

    for key in df['place_id'].unique(): # 미터ID 200개의 리스트를 unique()함수를 통해 추출합니다.
        temp = df.loc[df['place_id']==key] # 미터ID 하나를 할당합니다.
        temp_1h = temp.resample('1h').sum() # 1시간 단위로 정리합니다.
        temp_1day = temp.resample('D').sum() # 1일 단위로 정리합니다.

        # 시간별 예측
        model = ARIMA(temp_1h['target'], order=get_optimal_params(temp_1h['target'])) # AIC를 최소화하는 최적의 파라미터로 모델링합니다.
        results_ARIMA = model.fit(disp=-1)
        fcst = results_ARIMA.forecast(24) # 24시간을 예측합니다.
        for i in range(24):
            sub['X2018_7_1_'+str(i+1)+'h'] = [fcst[0][i]] # column명을 submission 형태에 맞게 지정합니다.

        # 일별 예측
        model = ARIMA(temp_1day['target'], order=get_optimal_params(temp_1day['target'])) # AIC를 최소화하는 최적의 파라미터로 모델링합니다.
        results_ARIMA = model.fit(disp=-1)
        fcst = results_ARIMA.forecast(10) # 10일을 예측합니다.
        for i in range(10):
            sub['X2018_7_'+str(i+1)+'_d'] = [fcst[0][i]] # column명을 submission 형태에 맞게 지정합니다.

        # 월별 예측
        # 일별로 예측하여 7월 ~ 11월의 일 수에 맞게 나누어 합산합니다.
        fcst = results_ARIMA.forecast(153)
        sub['X2018_7_m'] = [np.sum(fcst[0][:31])] # 7월 
        sub['X2018_8_m'] = [np.sum(fcst[0][31:62])] # 8월
        sub['X2018_9_m'] = [np.sum(fcst[0][62:92])] # 9월
        sub['X2018_10_m'] = [np.sum(fcst[0][92:123])] # 10월
        sub['X2018_11_m'] = [np.sum(fcst[0][123:153])] # 11월
        sub['meter_id'] = key 
        ret[key] = sub[submission.columns.tolist()]
        print(key)
        
    print('---- Modeling Done ----')
    return ret

In [110]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
submission_1002 = pd.read_csv("submission_1002.csv")
submission_1002 = submission_1002.set_index('meter_id')

In [111]:
test = test.set_index('Time')
test.index = pd.to_datetime(test.index)

In [ ]:
test = test['2017-08-25 00:00:00':]

# 1.Agregate by Time

In [113]:
temp = test.copy()
temp['hour'] = list(temp.reset_index()['Time'].apply(lambda x: x.hour))

In [131]:
for col in tqdm(test.columns):
    for t in range(24):
        data = temp.loc[temp['hour'] == t, col].copy()
        df = pd.DataFrame({
            'ds': data.index,
            'y': data.data,
        })

        ## Set model setting 
        m = Prophet(
            growth='linear', 
            changepoint_range=0.95,
            changepoint_prior_scale=0.1,
        )
        m.fit(df)

        ## Make dataframe for get a predicted value
        future = m.make_future_dataframe(periods=1, freq='T')

        ## Predict
        forecast = m.predict(future)
        
        ## Save the data
        submission.loc[col, 'X2017_7_1_' + str(t + 1) + '_h'] = forecast.loc[len(data), 'yhat']
        print(col)
        print(submission.loc[col, 'X2017_7_1_' + str(t + 1) + '_h'])
        print(submission.loc['X26', 'X2017_7_1_1_h'])
        submission
#     ## Show a graph
#     fig1 = m.plot(forecast)
#     plt.show()








  0%|          | 0/200 [00:00<?, ?it/s]INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


X26
1.132195875499108
1.132195875499108


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


KeyboardInterrupt: 

# 2. Aggregate by Day

In [116]:
test = test['2017-08-25 00:00:00':]
test_D = test.resample('D').sum()

In [117]:
for col in tqdm(test_D.columns):
    df = pd.DataFrame({
        'ds': test_D.index,
        'y': test_D[col],
    })

    ## Set model setting 
    m = Prophet(
        growth='linear', 
        changepoint_range=0.95,
        changepoint_prior_scale=0.1,
        yearly_seasonality=True
    )
    m.fit(df)
    
    ## Make dataframe for get a predicted value
    future = m.make_future_dataframe(periods=10)
    
    ## Predict
    forecast = m.predict(future)
    for i in range(10):
        submission.loc[col, 'X2018_7_' + str(i + 1) + '_d'] = forecast.loc[len(df) + i, 'yhat']
        
#     ## Show a graph
#     fig1 = m.plot(forecast)
#     plt.show()







  0%|          | 0/200 [00:00<?, ?it/s]INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.






  0%|          | 1/200 [00:01<05:15,  1.59s/it]INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.






  1%|          | 2/200 [00:03<05:05,  1.54s/it]INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.






  2%|▏         | 3/200 [00:04<04:57,  1.51s/it]INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.






  2%|▏         | 4/200 [00:05<04:51,  1.49s/it]INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.






  2%|▎         | 5/200 [00:07<04:44,  1.46s/it]INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.






  3%|▎         | 6/200 [00:08<04:49,  1.49s/it]INFO:fbprophet:Disabling 

# Aggregate by M

In [148]:
test = test['2017-08-25 00:00:00':]
test_M = test.resample('M').sum()

In [150]:
for col in tqdm(test.columns):
    df = pd.DataFrame({
        'ds': test_M.index,
        'y': test_M[col],
    })

    ## Set model setting 
    m = Prophet(
        growth='linear', 
        changepoint_range=0.8,
        changepoint_prior_scale=0.5,
    )
    m.fit(df)
    
    ## Make dataframe for get a predicted value
    future = m.make_future_dataframe(periods=5, freq='M')
    
    ## Predict
    forecast = m.predict(future)
    for i in range(5):
        submission.loc[col, 'X2018_' + str(i + 7) + '_m'] = forecast.loc[len(df) + i, 'yhat']
    
#     ## Show a graph
#     fig1 = m.plot(forecast)
#     plt.show()








  0%|          | 0/200 [00:00<?, ?it/s]INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 7.







  0%|          | 1/200 [00:01<06:04,  1.83s/it]INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 7.







  1%|          | 2/200 [00:03<06:01,  1.82s/it]INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasona

In [120]:
# test = to_panel_format(test) # 데이터 형식 변겅
# agg = prediction(test) # 예측

# output1 = pd.concat(agg, ignore_index=False)
# output2 = output1.reset_index().drop(['level_0','level_1'], axis=1)
# output2['id'] = output2['meter_id'].str.replace('X','').astype(int)
# output2 =  output2.sort_values(by='id', ascending=True).drop(['id'], axis=1).reset_index(drop=True)
# output2.to_csv('sub_baseline.csv', index=False)

In [121]:
submission.to_csv('sub_fbprophet.csv', index=False)

In [141]:
submission.columns

Index(['X2018_7_1_d', 'X2018_7_2_d', 'X2018_7_3_d', 'X2018_7_4_d',
       'X2018_7_5_d', 'X2018_7_6_d', 'X2018_7_7_d', 'X2018_7_8_d',
       'X2018_7_9_d', 'X2018_7_10_d', 'X2018_7_m', 'X2018_8_m', 'X2018_9_m',
       'X2018_10_m', 'X2018_11_m', 'X2017_7_1_1_h', 'X2017_7_1_2_h',
       'X2017_7_1_3_h', 'X2017_7_1_4_h', 'X2017_7_1_5_h', 'X2017_7_1_6_h',
       'X2017_7_1_7_h', 'X2017_7_1_8_h', 'X2017_7_1_9_h', 'X2017_7_1_10_h',
       'X2017_7_1_11_h', 'X2017_7_1_12_h', 'X2017_7_1_13_h', 'X2017_7_1_14_h',
       'X2017_7_1_15_h', 'X2017_7_1_16_h', 'X2017_7_1_17_h', 'X2017_7_1_18_h',
       'X2017_7_1_19_h', 'X2017_7_1_20_h', 'X2017_7_1_21_h', 'X2017_7_1_22_h',
       'X2017_7_1_23_h', 'X2017_7_1_24_h'],
      dtype='object')

In [144]:
submission = submission[['X2017_7_1_3_h', 'X2017_7_1_4_h', 'X2017_7_1_5_h', 'X2017_7_1_6_h',
       'X2017_7_1_7_h', 'X2017_7_1_8_h', 'X2017_7_1_9_h', 'X2017_7_1_10_h',
       'X2017_7_1_11_h', 'X2017_7_1_12_h', 'X2017_7_1_13_h', 'X2017_7_1_14_h',
       'X2017_7_1_15_h', 'X2017_7_1_16_h', 'X2017_7_1_17_h', 'X2017_7_1_18_h',
       'X2017_7_1_19_h', 'X2017_7_1_20_h', 'X2017_7_1_21_h', 'X2017_7_1_22_h',
       'X2017_7_1_23_h', 'X2017_7_1_24_h','X2018_7_1_d', 'X2018_7_2_d', 'X2018_7_3_d', 'X2018_7_4_d',
       'X2018_7_5_d', 'X2018_7_6_d', 'X2018_7_7_d', 'X2018_7_8_d',
       'X2018_7_9_d', 'X2018_7_10_d', 'X2018_7_m', 'X2018_8_m', 'X2018_9_m',
       'X2018_10_m', 'X2018_11_m']]

In [156]:
submission.reset_index().to_csv('sub_fbprophet.csv', index=False)

In [153]:
submission.head()

,X2017_7_1_3_h,X2017_7_1_4_h,X2017_7_1_5_h,X2017_7_1_6_h,X2017_7_1_7_h,X2017_7_1_8_h,X2017_7_1_9_h,X2017_7_1_10_h,X2017_7_1_11_h,X2017_7_1_12_h,...,X2018_7_6_d,X2018_7_7_d,X2018_7_8_d,X2018_7_9_d,X2018_7_10_d,X2018_7_m,X2018_8_m,X2018_9_m,X2018_10_m,X2018_11_m
meter_id,,,,,,,,,,,,,,,,,,,,,
X5,5.871975,5.811969,5.818463,5.517748,5.653051,6.294713,5.929817,6.191925,6.000700,6.118113,...,175.723503,177.448347,178.650011,178.743675,179.205640,6200.833349,7700.958922,9152.693346,10652.818918,12104.553343
X7,1.061695,1.108602,1.138722,1.097731,1.036900,1.833626,2.268283,1.951154,2.243338,2.875763,...,62.958806,65.360642,66.756898,66.093701,66.188261,1530.310469,1905.917707,2269.408582,2645.015820,3008.506695
X10,2.141522,2.287132,2.144392,2.176251,2.184234,4.515943,6.490085,5.513988,5.737400,6.243219,...,119.500375,124.355357,126.658505,116.424292,122.164499,3434.998308,4279.366037,5096.496098,5940.863828,6757.993889
X16,3.275182,3.279879,3.243610,3.362064,3.394243,3.529407,4.138677,5.165331,5.804338,6.195723,...,135.261440,138.446513,139.581565,137.489556,138.140799,4375.166168,5461.187042,6512.174985,7598.195859,8649.183802
X18,2.522369,2.517183,2.532031,2.554119,2.855331,2.691895,2.608116,2.562211,2.560741,2.581363,...,72.976400,73.759734,74.333166,74.286132,74.447586,2650.993820,3266.448779,3862.050351,4477.505310,5073.106883


In [155]:
submission.reset_index()

,meter_id,X2017_7_1_3_h,X2017_7_1_4_h,X2017_7_1_5_h,X2017_7_1_6_h,X2017_7_1_7_h,X2017_7_1_8_h,X2017_7_1_9_h,X2017_7_1_10_h,X2017_7_1_11_h,...,X2018_7_6_d,X2018_7_7_d,X2018_7_8_d,X2018_7_9_d,X2018_7_10_d,X2018_7_m,X2018_8_m,X2018_9_m,X2018_10_m,X2018_11_m
0,X5,5.871975,5.811969,5.818463,5.517748,5.653051,6.294713,5.929817,6.191925,6.000700,...,175.723503,177.448347,178.650011,178.743675,179.205640,6200.833349,7700.958922,9152.693346,10652.818918,12104.553343
1,X7,1.061695,1.108602,1.138722,1.097731,1.036900,1.833626,2.268283,1.951154,2.243338,...,62.958806,65.360642,66.756898,66.093701,66.188261,1530.310469,1905.917707,2269.408582,2645.015820,3008.506695
2,X10,2.141522,2.287132,2.144392,2.176251,2.184234,4.515943,6.490085,5.513988,5.737400,...,119.500375,124.355357,126.658505,116.424292,122.164499,3434.998308,4279.366037,5096.496098,5940.863828,6757.993889
3,X16,3.275182,3.279879,3.243610,3.362064,3.394243,3.529407,4.138677,5.165331,5.804338,...,135.261440,138.446513,139.581565,137.489556,138.140799,4375.166168,5461.187042,6512.174985,7598.195859,8649.183802
4,X18,2.522369,2.517183,2.532031,2.554119,2.855331,2.691895,2.608116,2.562211,2.560741,...,72.976400,73.759734,74.333166,74.286132,74.447586,2650.993820,3266.448779,3862.050351,4477.505310,5073.106883
5,X20,0.341951,0.323295,0.310171,0.319017,0.336039,0.356491,0.366478,0.370756,0.378022,...,66.667744,74.799130,81.284867,88.937208,95.824458,273.069319,342.775469,410.233034,479.939185,547.396750
6,X26,1.131848,1.121183,1.315696,1.644637,1.414368,1.250576,1.256359,1.338460,1.462564,...,21.819209,18.066308,14.821589,8.310950,6.214217,1470.789578,1804.822750,2128.080659,2462.113831,2785.371739
7,X30,0.505326,0.461974,0.487377,0.476496,0.483226,0.469662,0.456922,0.699107,4.894750,...,199.261383,216.495068,223.630435,239.736251,256.275005,4075.944716,5131.693024,6153.384936,7209.133245,8230.825156
8,X31,0.295246,0.298804,0.304771,0.467969,0.512772,0.554935,0.526508,0.572694,0.581480,...,13.112915,13.311944,13.421713,13.495527,13.511759,505.483206,621.106915,733.000828,848.624537,960.518449
9,X32,0.122283,0.129803,0.142175,0.516552,0.527597,0.523113,0.552709,0.666393,0.649005,...,13.302057,13.442888,13.395495,13.545294,13.480937,500.576815,614.801376,725.341273,839.565834,950.105732
